# Contagion Models and the Prediction of Legisign Replication

Today, we're going to learn how to build both simple and complex contagion simulation models using Python's [NDLib](https://ndlib.readthedocs.io/en/latest/reference/reference.html) (Network Diffusion Library) package, as well as plot the results of these simulations.

Once we're comfortable with NDLib, we're going to use these models to predict the increase and decrease in legisign replication of a social media trend (the "Tide Pod Challenge") using the "Kiki Challenge" trend to fit our model. We'll be using Google Trends data as our source of data to try to get after cross-platform social media interest across the Internet.

To install NDLib, run the following command on your terminal/command line:

`pip install ndlib`

Or, in your Jupyter notebook, you can run it like so (Warning: it can take a while to download/install):

In [1]:
!pip install ndlib

If you're using a Windows computer, you may need to separately download the file for the python-igraph package from [here](https://www.lfd.uci.edu/~gohlke/pythonlibs/#python-igraph) to successfully install the python-igraph package (a requirement for ndlib). The number after 'win' in the file name refers to whether you have a 32-bit or 64-bit Operating System (you can check this in your system settings in the "About" section. The number after 'cp' in the file name refers to your version of Python (3.7, 3.8, etc.), which you can figure out by running the command.

In [2]:
!python --version

Python 3.7.3


Once you have downloaded the correct file, put it in your local directory (or navigate to the file via the command line), and then run the command `pip install file_name`, replacing `file_name` with the name of the file that you downloaded. You should then be able to install NDLib as well via the aforementioned command `pip install ndlib`.

Now that we have all of our packages installed, let's import what we need into this session and get started!

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
from ndlib.utils import multi_runs
import ndlib.models.CompositeModel as gc
from ndlib.models.compartments import NodeThreshold
from ndlib.models.compartments import NodeStochastic
from scipy.optimize import minimize
import numpy as np
import seaborn as sns
import pandas as pd

# Function simulates model given parameters and gives me y_forecasted below (mean trend)
def simulate_net_diffusion(frac_infected = 0.01 ,threshold = 0.18, profile = 0.00001, p_removal = 0.017, num_exec = 10, num_iter=100, nproc=8):
    # Network generation
    g = nx.erdos_renyi_graph(1000, 0.1)
    
    # Composite Model instantiation
    sir_th_model = gc.CompositeModel(g)

    # Model statuses
    sir_th_model.add_status("Susceptible")
    sir_th_model.add_status("Infected")
    sir_th_model.add_status("Removed")

    # Compartment definition
    c1 = NodeThreshold(threshold=None, triggering_status="Infected")
    c2 = NodeStochastic(p_removal)

    # Rule definition
    sir_th_model.add_rule("Susceptible", "Infected", c1)
    sir_th_model.add_rule("Infected", "Removed", c2)

    # Model initial status configuration
    config = mc.Configuration()
    config.add_model_parameter('fraction_infected', frac_infected)

    # Setting nodes parameters
    for i in g.nodes():
        config.add_node_configuration("threshold", i, threshold)
        config.add_node_configuration("profile", i, profile)

    # Simulation execution
    sir_th_model.set_initial_status(config)
    iterations = sir_th_model.iteration_bunch(num_iter)

    trends = multi_runs(sir_th_model, execution_number=num_exec, iteration_number=num_iter, nprocesses=nproc)
    
    # convert into a dataframe that lists each number of infected nodes by iteration number (to make average calculation)
    df_infected = pd.DataFrame([execution['trends']['node_count'][1] for execution in trends])
    
    # Normalize each run: based on max value being the highest - normalize around this
    df_infected = df_infected.apply(lambda x: x/x.max(), axis=1)
    df_infected = pd.melt(df_infected, var_name='Execution', value_name='Infected')
    
    # Normalize (mean) values so that they are consistent with Google Trends Data for comparison:
    y_forecasted = df_infected.groupby('Execution').mean()* 100
    
    return y_forecasted

# Function returns results from all simulated runs for plotting 95% Confidence Intervals of Predictions
def full_simulate_net_diffusion(frac_infected = 0.01 ,threshold = 0.18, profile = 0.00001, p_removal = 0.017, num_exec = 10, num_iter=100, nproc=8):
    # Network generation
    g = nx.erdos_renyi_graph(1000, 0.1)
    
    # Composite Model instantiation
    sir_th_model = gc.CompositeModel(g)

    # Model statuses
    sir_th_model.add_status("Susceptible")
    sir_th_model.add_status("Infected")
    sir_th_model.add_status("Removed")

    # Compartment definition
    c1 = NodeThreshold(threshold=None, triggering_status="Infected")
    c2 = NodeStochastic(p_removal)

    # Rule definition
    sir_th_model.add_rule("Susceptible", "Infected", c1)
    sir_th_model.add_rule("Infected", "Removed", c2)

    # Model initial status configuration, assume 1% of population is infected
    config = mc.Configuration()
    config.add_model_parameter('fraction_infected', frac_infected)

    # Setting nodes parameters
    for i in g.nodes():
        config.add_node_configuration("threshold", i, threshold)
        config.add_node_configuration("profile", i, profile)

    # Simulation execution
    sir_th_model.set_initial_status(config)
    iterations = sir_th_model.iteration_bunch(num_iter)

    trends = multi_runs(sir_th_model, execution_number=num_exec, iteration_number=num_iter, nprocesses=nproc)
    
    # Convert into a dataframe that lists each number of infected nodes by iteration number (to make average calculation)
    df_infected = pd.DataFrame([execution['trends']['node_count'][1] for execution in trends])
    
    # Normalize each run, so that they are consistent with Google Trends Data for comparison:
    df_infected = df_infected.apply(lambda x: x/x.max(), axis=1)
    df_infected = pd.melt(df_infected, var_name='Execution', value_name='Infected')
    df_infected['Infected'] *= 100
    
    return df_infected

def mse(params, y_actual, time_steps):
    # Returns Mean Squared Error of the forecasted values, in comparison to actual values, for given parameters
    y_forecasted = simulate_net_diffusion(threshold=params[0], profile=params[1], p_removal=params[2], num_iter=time_steps)['Infected']
    y_forecasted.index = y_actual.index
    
    mse = ((y_forecasted - y_actual) ** 2).mean()
    return np.float(mse)

def optimize_net_diffusion_model(y_actual, time_steps, maxiter=1, params=[0.18, 0.00001, 0.017]): # arbitrarily chose initial parameters that were close enough in order to make this converge because need supercomputer to actually run
    # Parameters to optimize; passing in initial parameter values as a starting "guess": threshold, profile, p_removal - plugging in a ton of guesses and assessing how close they are to actual data (mean square framework)
    x0 = np.array(params)
        
    # Find parameters that minimize Mean Squared Error
    result = minimize(mse, x0 , args=(y_actual, time_steps), method='nelder-mead', options={'xtol': 1e-8, 'maxiter': maxiter, 'disp': True})
    
    return result

In [ ]:
# start with simple contagion model
# first need a network to create model
# we don't have a real network data so generate random network with random connections between nodes
g = nx.erdos_renyi_graph(1000, 0.1) # create a thousand nodes with certain probability of being connected by edges
# SI models: with some probability, will move from susceptible to infected if come into contact with infected node
si_model = ep.SIModel(g) # attach si model of cntagion to network from above
config = mc.Configuration()
config.add_model_parameter('beta', .001) # .001 probability that will become infected if come into contact with infected ndes
# set initial number of nodes to be infected at the beginning
config.add_model_parameter('fracton_infected', 0.05)
si_model.set_initial_status(config) # start with those parameters
# now let loose - set number of time steps for it to run
iterations = si_model.iteration_bunch(100) # 100 time steps
# combine into single trendline to describe how infectiousness is - how many people are infected over course of simulation
trends0 = si_model.build_trends(iterations)
# now plot w/ built in tool
viz = DiffusionTrend(si_model, trends0)
viz.plot()
# simple contagion is pretty linear and then slows down and eventually everyone is infected

##### complex contagion model over same random network we used

## classic threshold model from 1978 - when certain number of neighboring nodes are infected, I'm also going to be infected
th_model = ep.ThresholdModel(g) # attach threshold model to network
# configure it
config = mc.Configuration()
#set threshold to .1
threshold = .1
# use for loop to set threshold - if want to, can set diff thresholds for diff nodes
for n in g.nodes():
    config.add_node_configuration("threshold", n, threshold) # for each node, threshold will be threshold
    
config.add_model_parameter("fraction_infected", 0.05)
th_model.set_initial_status(config)
iterations = th_model.iteration_bunch(100)
trends1 = th_model.build_trends(iterations)
viz = DiffusionTrend(th_model, trends1)
viz.plot()
# more dramatic uptake in infection - dramatically increases to 100% infection - non-linear effects

### now diff. model where apply certain probability for each person after threshold is met (account for human agency)
pth_model = ep.ProfileThresholdModel(g)
config = mc.Configuration()
config.add_model_parameter("fraction_infected", 0.05) # 5% infected initially
threshold = 0.15
profile = 0.25  #probability of actually replicating legisign if infected
# also use for loop - also have possibility to have diff. profiles for diff. nodes
for i in g.nodes():
    config.add_node_configuration("threshold", i, threshold)
    config.add_node_configuration("profile", i, profile)
    
pth_model.set_intial_status(config)
iterations = pth_model.iteration_bunch(100)
trends2 = pth_model.build_trends(iterations)
viz = DiffusionTrend(pth_model, trends2)
viz.plot()
# no infections! nodes had agency and did not choose to replicate, even when infected

# when change initial infected to 10%, super quick infection
# more curved than original threshold model because probabilistic function that decides whether peple are crossing threshold

# remove parameter: to figure out when it's not poppular anymore and people are not adopting it anymore
sir_th_model = gc.CompositeModel(g) # special class that allows you to create any compartments you want (e.g. infected, susceptible, etc)
sir_th_model.add_status("Susceptible") # one compartment
sir_th_model.add_status("Infected")
sir_th_model.add_status("Removed")
# how to transition from one to another? need a probability. want a threshold for susceptible to infected
c1 = NodeThreshold(threshold = None, triggering_status="Infected") # if threshold goes over, will move to infected
# create stochastic node - if infected, some probability that move over to removed
c2 = NodeStochastic(0.05)
# above are probabilistic conditions that define transitions. now need to add rules to define what's going on
sir_th_model.add_rule('Susceptible', "Infected", c1) # when cross c1 threshold, move from susceptible to infected
sir_th_model.add_rule("Infected", "Removed", c2)
# bring in material from above - from config to viz.plot


# these are all single simulations (with multiple time steps within it) - if run multiple times, can find 95% CI and stuff
trends = multi_runs(si_model, execution_number = 5, iteration_number = 100) # run five times (using previous simple model)
viz = Diffusiontrend(si_model, trends)
viz.plot(percentile = .95) # set percentile that you want - 95% CI
# have confidence interval, have mean prediction



###### predict tide pod challenge contagion
# want to have many samples, train model using many similar contagion processes - takes a lot of time so just using kiki challenge
# look at google trends data - these numbers are arbitrary, close enough for lack of processing power
simulated_results_kiki = full_simulated_net_diffusion(frac_infected = 0.01, threshold = 0.038, profile = 0.0000105,
                                                     p_removal = 0.22, num_iter = 32, num_exec = 20)
# num_iter = 32 because look at where pattern started (first one)
# load in data run from top
kiki_challenge = pd.read_csv('kiki_challenge_trend.csv')
# select where to start and stop
kiki_challenge = kiki_challene.loc['2018-07-01': '2019-02-03']
#rename column
kiki_challenge.columns = ['num_infected']
# turn numbers into strings and remove < (less than 1)
kiki_challenge = pd.to_numeric(kik_challenge['num_infected'].str.strip("<"))
plt.figure(figsize=(10,5))
sns.lineplot(x = 'Execution', y = 'Infected', data = simulated_results_kiki, label = 'Simulated Results') # use seaborn because easy to make 95% CI around line plot
# superimpose line of actual trend on top of predicted trend line using matplotlib
plt.plot(kiki_challenge, label = "Actual Kiki Challenge Trend")
plt.sticks(rotation = 90)
plt.title .....

# now check and see how it does with tide pod data
tide_pod_challenge = pd.read_csv('tide_pod"challenge_trend.csv')
tide_pod_challenge = tide_pod_challenge.loc['2017-12-17': '2018-07-22'] # locate where is starts and ends
tide_pod_challenge.columns = ['num_infected']
tide_pod_challenge = pd.to_numeric(tide_pod_challenge['num_infected'].str.strip("<"))
plt.figure(figsize = 10,5)
sns.lineplot(x = "Execution", y = "Infected", data = simulated_results_kiki, label = "Prediction")
plt.plot(tide_pod_challene, label = "Actual Tide Pod Challenge Trend")
